##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/examples/blob/master/courses/udacity_intro_to_tensorflow_for_deep_learning/l06c03_exercise_flowers_with_transfer_learning_solution.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/examples/blob/master/courses/udacity_intro_to_tensorflow_for_deep_learning/l06c03_exercise_flowers_with_transfer_learning_solution.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

# TensorFlow Hub

[TensorFlow Hub](http://tensorflow.org/hub)は、使用できる既にトレーニング済みのTensorFlowモデルのオンラインリポジトリです。
これらのモデルは、そのまま使用することも、Transfer Learningに使用することもできます。

転移学習は、既存のトレーニング済みモデルを取得し、それを拡張して追加の作業を行うプロセスです。 これには、可能な出力の異なるセットを取得するために、モデルの大部分を変更せずに残し、最終層を追加および再トレーニングすることが含まれます。

ここでは、[TensorFlow Module Hub](https://tfhub.dev/)で利用可能なすべてのモデルを見ることができます。

このColabを開始する前に、上のメニューから[ランタイム->すべてのランタイムをリセット...]を選択して、Colab環境をリセットする必要があります。


# Imports



以前見たいくつかの通常のインポートです。 新しいものは、このColabが多用するtensorflow_hubをインポートしています。

In [1]:
#from __future__ import absolute_import, division, print_function, unicode_literals

In [2]:
# try:
#   # Use the %tensorflow_version magic if in colab.
#   %tensorflow_version 2.x
# except Exception:
#   pass

import tensorflow as tf

In [3]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow_hub as hub
import tensorflow_datasets as tfds

from tensorflow.keras import layers

In [4]:
import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

# TODO: Download the Flowers Dataset using TensorFlow Datasets

以下のセルでは、TensorFlow Datasetsを使用してFlowersデータセットをダウンロードします。 [TensorFlow Datasets documentation](https://www.tensorflow.org/datasets/datasets#tf_flowers)を見ると、Flowersデータセットの名前が `tf_flowers`であることがわかります。 また、このデータセットはTRAININGセットにのみ分割されていることがわかります。 したがって、このトレーニングセットを「training_set」と「validation_set」に分割するには、「tfds.splits」を使用する必要があります。 70%が `training_set`に対応し、30%が` validation_set`に対応するように `[70、30]`分割を行います。 次に、「tfds.load」を使用して「tf_flowers」データセットをロードします。 `tfds.load`関数が必要なすべてのパラメーターを使用していることを確認し、データセットに関する情報を取得できるようにして、データセットに関する情報を取得できるようにします。


In [5]:
splits = tfds.Split.TRAIN.subsplit([70, 30])

(training_set, validation_set), dataset_info = tfds.load('tf_flowers', with_info=True, as_supervised=True, split=splits)

Dataset tf_flowers downloaded and prepared to /Users/hironsuz/tensorflow_datasets/tf_flowers/1.0.0. Subsequent calls will reuse this data.


# TODO: Print Information about the Flowers Dataset

データセットをダウンロードしたら、データセット情報を使用してデータセット内のクラスの数を印刷し、トレーニングセットと検証セットにある画像の数をカウントするコードを記述します。

In [6]:
num_classes = dataset_info.features['label'].num_classes

num_training_examples = 0
num_validation_examples = 0

for example in training_set:
  num_training_examples += 1

for example in validation_set:
  num_validation_examples += 1

print('Total Number of Classes: {}'.format(num_classes))
print('Total Number of Training Images: {}'.format(num_training_examples))
print('Total Number of Validation Images: {} \n'.format(num_validation_examples))

Total Number of Classes: 5
Total Number of Training Images: 2590
Total Number of Validation Images: 1080 



Flowersデータセットの画像は、すべて同じサイズではありません。

In [7]:
for i, example in enumerate(training_set.take(5)):
  print('Image {} shape: {} label: {}'.format(i+1, example[0].shape, example[1]))

Image 1 shape: (240, 320, 3) label: 4
Image 2 shape: (375, 500, 3) label: 1
Image 3 shape: (333, 500, 3) label: 1
Image 4 shape: (227, 320, 3) label: 4
Image 5 shape: (246, 320, 3) label: 1


# TODO: Reformat Images and Create Batches

下のセルで、すべての画像をMobileNet v2（224、224）で期待される解像度に再フォーマットし、それらを正規化する関数を作成します。 関数は、引数として「イメージ」と「ラベル」を取り、新しい「イメージ」と対応する「ラベル」を返す必要があります。 次に、サイズが32のトレーニングバッチと検証バッチを作成します。

In [8]:
IMAGE_RES = 224

def format_image(image, label):
  image = tf.image.resize(image, (IMAGE_RES, IMAGE_RES))/255.0
  return image, label

BATCH_SIZE = 32

train_batches = training_set.shuffle(num_training_examples//4).map(format_image).batch(BATCH_SIZE).prefetch(1)

validation_batches = validation_set.map(format_image).batch(BATCH_SIZE).prefetch(1)

# Do Simple Transfer Learning with TensorFlow Hub

次に、TensorFlow Hubを使用してTransfer Learningを実行しましょう。 転移学習では、すでに訓練されたモデルの一部を再利用し、モデルの最終層または複数の層を変更してから、独自のデータセットでそれらの層を再訓練します。

### TODO: Create a Feature Extractor
以下のセルで、MobileNet v2を使用して「feature_extractor」を作成します。 TensorFlow Hubからの部分モデル（最終分類レイヤーなし）は、特徴ベクトルと呼ばれることに注意してください。 [TensorFlow Hubドキュメント](https://tfhub.dev/s?module-type=image-feature-vector&q=tf2)にアクセスして、利用可能な機能ベクトルのリストを確認します。 「tf2-preview / mobilenet_v2 / feature_vector」をクリックします。 ドキュメントを読み、対応する「URL」を取得して、MobileNet v2機能ベクトルを取得します。 最後に、正しい「input_shape」パラメーターで「hub.KerasLayer」を使用して、「feature_extractor」を作成します。

In [9]:
URL = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
feature_extractor = hub.KerasLayer(URL,
                                   input_shape=(IMAGE_RES, IMAGE_RES, 3))

### TODO: Freeze the Pre-Trained Model

下のセルで、特徴抽出レイヤーの変数をフリーズします。これにより、トレーニングでは最終分類レイヤーのみが変更されます。

In [10]:
feature_extractor.trainable = False

### TODO: Attach a classification head

以下のセルで「tf.keras.Sequential」モデルを作成し、事前トレーニング済みのモデルと新しい分類レイヤーを追加します。 分類レイヤーには、Flowersデータセットと同じ数のクラスが必要であることに注意してください。 最後に、シーケンシャルモデルの概要を印刷します。

In [11]:
model = tf.keras.Sequential([
  feature_extractor,
  layers.Dense(num_classes, activation='softmax')
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 1280)              2257984   
_________________________________________________________________
dense (Dense)                (None, 5)                 6405      
Total params: 2,264,389
Trainable params: 6,405
Non-trainable params: 2,257,984
_________________________________________________________________


### TODO: Train the model

セルベローズトレインでは、他のモデルと同様に、最初に「コンパイル」を呼び出し、次に「フィット」を呼び出します。 両方の方法を適用するときは、適切なパラメーターを使用してください。 モデルを6エポックのみトレーニングします。

In [12]:
model.compile(
  optimizer='adam',
  loss='sparse_categorical_crossentropy',
  metrics=['accuracy'])

EPOCHS = 6

history = model.fit(train_batches,
                    epochs=EPOCHS,
                    validation_data=validation_batches)

Epoch 1/6
81/81 [==============================] - 60s 742ms/step - loss: 0.7750 - accuracy: 0.7158


KeyboardInterrupt: 

6エポックのトレーニングで最大88％の検証精度が得られることがわかります。 これは、前のレッスンで作成したモデルを大幅に改善したもので、80エポックのトレーニングで最大76％の精度を得ることができました。 この違いの理由は、MobileNet v2が専門家によって長い間慎重に設計され、大規模なデータセット（ImageNet）でトレーニングされたためです。

# TODO: Plot Training and Validation Graphs

下のセルに、トレーニングと検証の精度/損失グラフをプロットします。

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(EPOCHS)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

ここで少し興味深いのは、実行の開始から終了まで、検証パフォーマンスがトレーニングパフォーマンスよりも優れていることです。

この理由の1つは、検証パフォーマンスはエポックの終わりに測定されるが、トレーニングパフォーマンスはエポック全体の平均値であるということです。

もっとも大きな理由は、すでにFlower画像でトレーニングされたMobileNetの大部分を再利用していることです。

# TODO: Check Predictions

以下のセルで、データセット情報からラベル名を取得し、それらをNumPy配列に変換します。 配列を印刷して、ラベル名が正しいことを確認してください。

In [ ]:
class_names = np.array(dataset_info.features['label'].names)

print(class_names)

### TODO: Create an Image Batch and Make Predictions

以下のセルで、「next（）」関数を使用して「image_batch」とそれに対応する「label_batch」を作成します。 `.numpy（）`メソッドを使用して、 `image_batch`と` label_batch`の両方をnumpy配列に変換します。 次に、 `.predict（）`メソッドを使用して、モデルを介して画像バッチを実行し、予測を行います。 次に、 `np.argmax（）`関数を使用して、各画像の最適な予測のインデックスを取得します。 最後に、最良の予測のインデックスをクラス名に変換します。

In [ ]:
image_batch, label_batch = next(iter(train_batches))


image_batch = image_batch.numpy()
label_batch = label_batch.numpy()

predicted_batch = model.predict(image_batch)
predicted_batch = tf.squeeze(predicted_batch).numpy()

predicted_ids = np.argmax(predicted_batch, axis=-1)
predicted_class_names = class_names[predicted_ids]

print(predicted_class_names)

### TODO: Print True Labels and Predicted Indices

下のセルに、実際のラベルと予測ラベルのインデックスを印刷します。

In [ ]:
print("Labels:           ", label_batch)
print("Predicted labels: ", predicted_ids)

# Plot Model Predictions

In [ ]:
plt.figure(figsize=(10,9))
for n in range(30):
  plt.subplot(6,5,n+1)
  plt.subplots_adjust(hspace = 0.3)
  plt.imshow(image_batch[n])
  color = "blue" if predicted_ids[n] == label_batch[n] else "red"
  plt.title(predicted_class_names[n].title(), color=color)
  plt.axis('off')
_ = plt.suptitle("Model predictions (blue: correct, red: incorrect)")

# TODO: Perform Transfer Learning with the Inception Model

[TensorFlow Hubドキュメント](https://tfhub.dev/s?module-type=image-feature-vector&q=tf2)に移動し、「tf2-preview / inception_v3 / feature_vector」をクリックします。 この特徴ベクトルは、Inception v3モデルに対応しています。 以下のセルで、Transfer Learningを使用して、Inception v3を事前学習済みモデルとして使用してCNNを作成し、Flowersデータセットの画像を分類します。 Inceptionは、入力として299 x 299ピクセルの画像を受け取ることに注意してください。 Inception v3で得られる精度とMobileNet v2で得られる精度を比較します。

In [ ]:
IMAGE_RES = 299

(training_set, validation_set), dataset_info = tfds.load('tf_flowers', with_info=True, as_supervised=True, split=splits)
train_batches = training_set.shuffle(num_training_examples//4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_set.map(format_image).batch(BATCH_SIZE).prefetch(1)

URL = "https://tfhub.dev/google/tf2-preview/inception_v3/feature_vector/4"
feature_extractor = hub.KerasLayer(URL,
  input_shape=(IMAGE_RES, IMAGE_RES, 3),
  trainable=False)

model_inception = tf.keras.Sequential([
  feature_extractor,
  tf.keras.layers.Dense(num_classes, activation='softmax')
])

model_inception.summary()

In [ ]:
model_inception.compile(
  optimizer='adam', 
  loss='sparse_categorical_crossentropy',
  metrics=['accuracy'])

EPOCHS = 6

history = model_inception.fit(train_batches,
                    epochs=EPOCHS,
                    validation_data=validation_batches)